In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of windmark.core.structs failed: Traceback (most recent call last):
  File "/home/grantham/.cache/pypoetry/virtualenvs/windmark-gThMBSDt-py3.11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/grantham/.cache/pypoetry/virtualenvs/windmark-gThMBSDt-py3.11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/grantham/.pyenv/versions/3.11.8/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/grantham/windmark/windmark/core/structs.py", line 76, in <module>
    class Hyperparameters(pydantic.BaseModel):
  File "/home/gr

In [26]:
from windmark.core.structs import Schema

In [24]:
schema = Schema(
    use_chip="discrete",
    merchant_state="discrete",
    # merchant_city="discrete",
    # merchant_name="entity",
    # mcc="discrete",
    # amount="continuous",
    # timedelta="continuous",
    # timestamp="temporal",
)

In [1]:
cadence = 8

for i in range(100):
    if i % cadence == 0:
        print(i)

0
8
16
24
32
40
48
56
64
72
80
88
96


In [15]:
L = 192
F = 12

In [16]:
(L * F) ** 2

5308416

In [19]:
L * F * F + L * L * F

470016

In [20]:
5308416 / 470016

11.294117647058824